# Laboratorio 7

1. Utilice los juicios de relevancia de la colección CACM almacenados en el archivo qrels.txt
1. Utilice la salida de la práctica 6 para calcular Precisión, Recuerdo, Media F, Precisión R a Z documentos para cada una de las consultas. Z será un valor leído de pantalla donde se establecerá a que número de documentos deberán calcularse la métricas solicitadas.
1. Guarde sus resultados en un archivo con Z = 20 en el formato siguiente:

|  Consulta |Precisión|Recuerdo| Medida F  | Precisión R  |
|:---:|:---:|:---:|:---:|:---:|
|  1    | 0.45  | 0.35  | 0.39  | 0.43 |
| 2     | 0.5   | 0.4   | 0.2   | 0.45 | 
    

---



Primero, importaremos el archivo qrels.text como un dataframe con pandas


In [262]:
import pandas as pd
rel_df = pd.read_csv("assets/qrels.text", sep=" ", header=None, usecols=[0,1], names=['Query_num','Doc_num'])
rel_df['Relevant'] = 1
print(rel_df)

     Query_num  Doc_num  Relevant
0            1     1410         1
1            1     1572         1
2            1     1605         1
3            1     2020         1
4            1     2358         1
..         ...      ...       ...
791         63     2714         1
792         63     2973         1
793         63     3075         1
794         63     3156         1
795         64     2651         1

[796 rows x 3 columns]


Ahora, importamos los juicios de coincidencia creados en la práctica 6

In [263]:
import json
with open('assets\\CACM_TF_rels.txt', 'r') as file:
    sim_df = pd.read_json(file)
# change names of sim_df
sim_df = sim_df.set_axis(['Query_num','Doc_num','Sim'], axis=1)
#sim_df['Relevant'] = 0
print(sim_df)


       Query_num  Doc_num       Sim
0              1     2371  0.594818
1              1     1071  0.591864
2              1     1938  0.575613
3              1     2319  0.541313
4              1     1680  0.517697
...          ...      ...       ...
71429         64     2628  0.037037
71430         64     2314  0.036442
71431         64     1470  0.031879
71432         64     1155  0.030179
71433         64     1665  0.026978

[71434 rows x 3 columns]


In [264]:
# contamos en df similitudes (recuperados)
cont_sim_df = pd.DataFrame( )
for i in range(1,64+1):
    cont_sim_df[i] = sim_df[sim_df.Query_num == i].count()
    #print(sim_df[sim_df.Query_num == i].count())
    
cont_sim_df.drop(['Doc_num','Sim'], inplace=True)
cont_sim_df = cont_sim_df.T
cont_sim_df

,Query_num
1,1393
2,144
3,548
4,1280
5,1069
...,...
60,1338
61,956
62,615
63,1973


In [265]:
# contamos en df relevantes
cont_rel_df = pd.DataFrame( )
for i in range(1,64+1):
    cont_rel_df[i] = rel_df[rel_df.Query_num == i].count()
    #print(sim_df[sim_df.Query_num == i].count())
    
cont_rel_df.drop(['Doc_num','Relevant'], inplace=True)
cont_rel_df = cont_rel_df.T
cont_rel_df

,Query_num
1,5
2,3
3,6
4,12
5,8
...,...
60,27
61,31
62,8
63,12


In [266]:
# df relevantes recuperados
relRec_df_all = pd.merge(sim_df, rel_df, how='left', on=['Query_num','Doc_num'])
relRec_df = relRec_df_all.loc[relRec_df_all.Relevant == 1]
#relRec_df.drop(['Doc_num','Sim','Relevant'],inplace=True)
relRec_df

,Query_num,Doc_num,Sim,Relevant
8,1,1572,0.470898,1.0
37,1,1605,0.405648,1.0
105,1,2358,0.329774,1.0
152,1,1410,0.293151,1.0
328,1,2020,0.209061,1.0
...,...,...,...,...
70069,63,3075,0.241523,1.0
70174,63,3156,0.223607,1.0
70177,63,2664,0.222222,1.0
70181,63,1601,0.220366,1.0


In [267]:
# contamos df relevantes recuperados
# contamos en df relevantes
cont_relRec_df = pd.DataFrame( )
for i in range(1,64+1):
    cont_relRec_df[i] = relRec_df[relRec_df.Query_num == i].count()
    #print(sim_df[sim_df.Query_num == i].count())
    
cont_relRec_df.drop(['Doc_num','Sim','Relevant'], inplace=True)
cont_relRec_df = cont_relRec_df.T
cont_relRec_df

,Query_num
1,5
2,0
3,5
4,11
5,8
...,...
60,21
61,28
62,7
63,11


Ahora, con los conteos hechos, calculamos las estadísticas sobre cada query.

Todo se guardará sobre un nuevo dataframe

In [268]:
# antes, agrupamos para contar los primeros Z=20 records de cada recuperación
Z = 20
Z_20_df = pd.DataFrame()
relRec_df_all['Relevant'].fillna(0, inplace=True)
for i in range(1,64+1):
    twenty = relRec_df_all[relRec_df_all['Query_num']==i].iloc[:Z]
    Z_20_df[i] = twenty[twenty['Relevant'] == 1].count()


In [269]:
#twenty = relRec_df_all[relRec_df_all['Query_num']==1].iloc[:20]
#
# twenty
Z_20_df.drop(['Doc_num','Sim','Relevant'], inplace=True)
Z_20_df = Z_20_df.T
Z_20_df


,Query_num
1,1
2,0
3,1
4,3
5,2
...,...
60,1
61,7
62,0
63,1


In [270]:
# precision:            #docs_rel_rec/#docs_recuperados
# recall (recuerdo):    #docs_rel_rec/#docs_rel
# F-score:              2PR/(P+R)
# Presicion R a Z:                  Precisión hasta el Z-ésimo documento recuperado (Z=20)

stats_df = pd.DataFrame({'Precision': pd.Series(dtype='float'), 
                         'Recall': pd.Series(dtype='float'),
                         'F-score': pd.Series(dtype='float'),
                         'Precision-20': pd.Series(dtype='float')})
for i in range(1,64+1):
    pres = (cont_relRec_df.at[i,'Query_num'])/(cont_sim_df.at[i,'Query_num'])
    recall = (cont_relRec_df.at[i,'Query_num'])/(cont_rel_df.at[i,'Query_num'])
    if (pres + recall == 0):
        F_score = 0
    else:
        F_score = (2*pres*recall)/(pres+recall)
    pres_RZ =  (Z_20_df.at[i, 'Query_num']) / 20
    stats_df.loc[len(stats_df.index)+1] = [pres, recall, F_score, pres_RZ]
stats_df.fillna(0, inplace=True)


C:\Users\poncho\AppData\Local\Temp\ipykernel_9092\969746051.py:12: RuntimeWarning: invalid value encountered in longlong_scalars
  recall = (cont_relRec_df.at[i,'Query_num'])/(cont_rel_df.at[i,'Query_num'])
C:\Users\poncho\AppData\Local\Temp\ipykernel_9092\969746051.py:12: RuntimeWarning: invalid value encountered in longlong_scalars
  recall = (cont_relRec_df.at[i,'Query_num'])/(cont_rel_df.at[i,'Query_num'])
C:\Users\poncho\AppData\Local\Temp\ipykernel_9092\969746051.py:12: RuntimeWarning: invalid value encountered in longlong_scalars
  recall = (cont_relRec_df.at[i,'Query_num'])/(cont_rel_df.at[i,'Query_num'])
C:\Users\poncho\AppData\Local\Temp\ipykernel_9092\969746051.py:12: RuntimeWarning: invalid value encountered in longlong_scalars
  recall = (cont_relRec_df.at[i,'Query_num'])/(cont_rel_df.at[i,'Query_num'])
C:\Users\poncho\AppData\Local\Temp\ipykernel_9092\969746051.py:12: RuntimeWarning: invalid value encountered in longlong_scalars
  recall = (cont_relRec_df.at[i,'Query_num'

In [275]:
# verificar datos
display(stats_df.iloc[47])
display(cont_rel_df.iloc[47])
display(cont_sim_df.iloc[47])
display(cont_relRec_df.iloc[47])
display(Z_20_df.iloc[47])

Precision       0.003475
Recall          0.750000
F-score         0.006918
Precision-20    0.100000
Name: 48, dtype: float64

Query_num    12
Name: 48, dtype: int64

Query_num    2590
Name: 48, dtype: int64

Query_num    9
Name: 48, dtype: int64

Query_num    2
Name: 48, dtype: int64

Por último, lo guardamos a un archivo

In [272]:
from pathlib import Path  
filepath = Path('results/stats.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
stats_df.to_csv(filepath) 
filepath = Path('results/stats.json')  
stats_df.to_json(filepath) 